# Discrete LTI Systems

这是陈硕写的《数字信号处理入门实验》的第一个实验，介绍离散线性时不变（缩写 DLTI）系统的基本性质。
最新版网址： http://github.com/chenshuo/notes

如果想要执行交互式的内容，可以用 Colab 打开：
https://colab.research.google.com/github/chenshuo/notes/blob/master/dsp_labs/1-dlti.ipynb

本章内容的视频讲解在
1. [移动平均](https://youtu.be/kTlJMCUknF8) 国内：[移动平均](https://www.bilibili.com/video/BV1Qd4y1Q7e4)
1. [自回归](https://youtu.be/NXnkhopsUnE) 国内：[自回归](https://www.bilibili.com/video/BV1FW4y1U7dY)

首先引入 NumPy、SciPy.signal、Matplotlib、LibROSA 等必要的库。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

import librosa as rosa
import librosa.display
from IPython.display import Audio

from ipywidgets import interact

np.set_printoptions(suppress=True)
rosa.version.show_versions()

定义两个常用的函数，用于画零极点图和输入输出对比图。

In [ ]:
def draw_pzmap(dlti):
  plt.plot(np.real(dlti.zeros), np.imag(dlti.zeros), 'o', mfc='none')
  plt.plot(np.real(dlti.poles), np.imag(dlti.poles), 'x')
  w = np.linspace(0, 2*np.pi)
  plt.plot(np.cos(w), np.sin(w), 'y--')
  limits = plt.axis("equal")
  plt.grid()

def draw_resp_stem(x, y):
  plt.stem(x, use_line_collection=True)
  (markerline, _, _) = plt.stem(y, linefmt='r', markerfmt='ro', use_line_collection=True)
  markerline.set_markerfacecolor('none')

查看一下 Jupyter 的版本。

In [ ]:
!jupyter --version

## DLTI 的两种基本形式：FIR 与 IIR

DLTI 系统有两种基本的形式，通常的名字是 FIR 与 IIR，主要区别是是系统否有记忆，即当前输出是否与过去的输出有关，还是只与输入有关。

FIR 其实就是加权移动平均（Weighted Moving Average, WMA for short）。

![FIR](data/fir.png)

$$\begin{aligned}
  y[n] &= \sum_{i=0}^{L}b[i] x[n-i] \\
       & = b[0]x[n] + b[1]x[n-1] + \cdots + b[L]x[n-L]
\end{aligned} $$


根据以上定义我们可以很容易写出自己的 WMA 实现，而 SciPy 里也有多种办法计算 WMA，包括

* [scipy.signal.convolve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html), [oaconvolve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.oaconvolve.html), [fftconvolve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.fftconvolve.html)
* [scipy.signal.lfilter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html)
* [scipy.signal.dlsim](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.dlsim.html)
* [scipy.signal.dlti.output](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.dlti.output.html)

```{toggle}
FIR 其实也可以看成是最简单的前向神经网络，没有 bias，也没有 activation function，输出完全是输入按不同权重的线性组合。
```

WMA 的实现是两重循环，内循环负责加权求和，外循环负责移动。
以下代码的“加权求和”这一步用 `NumPy.dot()` 实现，比较简洁。
`dot()` 是计算两个向量的点积，也叫内积。

In [ ]:
from typing import Sequence

def direct_wma(b : Sequence, x : Sequence) -> Sequence:
  y = np.zeros(len(x))
  x_pad = np.concatenate((np.zeros(len(b)-1), x))
  b_rev = np.array(b[::-1], dtype='d')
  for i in range(len(x)):
    y[i] = np.dot(b_rev, x_pad[i : i + len(b)])
  return y

用我们自己实现的 `direct_wma()` 来计算 $y[n] = 2x[n] + x[n-1]$.

In [ ]:
b = [2, 1]
x = np.arange(10, dtype='d')
y = direct_wma(b, x)
print(np.array([x, y]).T)

计算结果和 `scipy.signal.lfilter()` 是一致的。

In [ ]:
# b 和 x 同上
y = signal.lfilter(b, [1], x)
print(np.array([x, y]).T)

当然 `direct_wma()` 不是什么高效的实现，只是展示一下基本编程方法。

第二种形式，IIR，是在 FIR 的基础上，把过去的输出也用来计算新的输出。

![IIR](data/iir.png)

这种形式过去也叫梯子（ladder），或格子（lattice），是比较形象的说法。

$$\begin{aligned}
y[n] = &\sum_{i=0}^{L}b[i]x[n-i] - \sum_{i=1}^{M}a[i\,]y[n-i\,] \\
 = & \  b[0]x[n] + b[1]x[n-1] + \cdots + b[L]x[n-L] \\
        &-(a[1]y[n-1]+a[2]y[n-2]+\cdots+a[M]y[n-M])
\end{aligned}$$


SciPy 里提供了多种实现，本文以下主要使用第一个 `scipy.signal.lfilter()` 来计算输出，而用第三个 `scipy.signal.dlti` class 来分析离散线性时不变系统（频率响应、冲击响应、零极点等等）。

* [scipy.signal.lfilter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html)
* [scipy.signal.dlsim](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.dlsim.html)
* [scipy.signal.dlti.output](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.dlti.output.html)


有的领域把这种形式叫“自回归” autoregressive（特指用到当前输入和过去的输出），
如果同时用到了过去的输入和过去的输出，那和前面的移动平均合在一起，叫 [ARMA 模型](https://en.wikipedia.org/wiki/Autoregressive%E2%80%93moving-average_model)。

```{toggle}
这也可以看成最简单的循环神经网络（RNN），没有 bias，也没有 activation function，只有两个权重。
```

我们自己实现 ARMA 也不难，用两个循环分别计算出第 1 项 sum 和第 2 项 sum，然后相减就得到了 $y[n]$。
稍有挑战的是如果输入是 Iterable，而不是 Sequence，那么该如何实现？
正式的系统一般会用循环缓冲区（circular buffer），我们自己练手写的话不妨粗放一些，直接用数组的

## Moving average

Julius O. Smith, _Introduction to Digital Filters with Audio Applications_, http://ccrma.stanford.edu/~jos/filters/

Chapter 1: https://ccrma.stanford.edu/~jos/filters/Simplest_Lowpass_Filter.html

### $N=2$

$y[n] = \dfrac{1}{2}x[n]+\dfrac{1}{2}x[n-1]$

In [ ]:
b = [1/2, 1/2]

x = np.arange(11)
y = signal.lfilter(b, [1], x)  # or signal.convolve(b, x)
print(np.array([x, y]).T)

[scipy.signal.dlti](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.dlti.html) Discrete-time linear time invariant system

In [ ]:
dlti = signal.dlti(b, [1])
print('Zeros:', dlti.zeros)
print('Poles:', dlti.poles)
draw_pzmap(dlti)

In [ ]:
w, mag = dlti.freqresp()
plt.plot(w, np.abs(mag))

In [ ]:
w, mag, phase = dlti.bode()
plt.semilogx(w/np.pi, mag)
plt.figure(2)
plt.plot(w/np.pi,phase)

In [ ]:
w = np.arange(20) * np.pi
x = np.cos(w)  # [1, -1, 1, -1, 1, -1, ...]
y = np.convolve(b, x)

draw_resp_stem(x, y)
#plt.stem(x, use_line_collection=True)
#(marker, _, _) = plt.stem(y, linefmt='r', markerfmt='ro', use_line_collection=True)
#marker.set_markerfacecolor('none')

In [ ]:
@interact(f = (0, 1, 0.02))
def resp(f = 0.5):
  t = np.linspace(0, 1, 101)
  x = np.cos(t * 2*np.pi * f * 100)
  plt.plot(t, x)
  y = signal.lfilter([1/2, 1/2], [1], x)
  plt.plot(t, y)
  plt.show()

In [ ]:
fft = np.fft.rfft([1/2, 1/2], n=64)  # zero padding
(marker, _, _) = plt.stem(np.abs(fft), use_line_collection=True)
marker.set_markerfacecolor('none')

fr = dlti.freqresp(w=np.linspace(0, 1, len(fft)) * np.pi)[1]
plt.plot(np.abs(fr), 'r--')

### $N = 3$

$y[n] = \dfrac{1}{3}x[n] + \dfrac{1}{3}x[n-1] + \dfrac{1}{3}x[n-2]$

In [ ]:
b3 = [1/3, 1/3, 1/3]
dlti3 = signal.dlti(b3, [1])

w, mag = dlti3.freqresp()
plt.plot(w/np.pi, np.abs(mag))
print('min_mag = %.2f dB' % (20 * np.log10(np.min(np.abs(mag)))))

In [ ]:
w, mag, phase = dlti3.bode()
plt.plot(w/np.pi, mag)

In [ ]:
draw_pzmap(dlti3)
print(np.angle(dlti3.zeros)/np.pi*180)

In [ ]:
t = np.arange(20)
x = np.cos(t * 2/3 * np.pi)
y = np.convolve(x, b3)

draw_resp_stem(x, y)
print(x[0:10])

In [ ]:
t = np.arange(20)
x = np.sin(t * 2/3 * np.pi)
y = np.convolve(x, b3)
draw_resp_stem(x, y)

# x = [1, -1, 0, 1, -1, 0, 1, -1, 0, 1]

In [ ]:
@interact(phase=(0, 90, 5))
def resp(phase=45):
  t = np.arange(15)
  x = np.cos(t * 2/3 * np.pi - phase / 180 * np.pi)
  y = signal.lfilter(b3, [1], x)
  plt.stem(t, x, use_line_collection=True)
  t1 = np.arange(141) / 10
  plt.plot(t1, np.cos(t1 * 2/3 * np.pi - phase / 180 * np.pi), 'y--')
  (markerline, stemlines, baseline) = plt.stem(t, y, linefmt='r', markerfmt='ro', use_line_collection=True)
  markerline.set_markerfacecolor('none')

### $N=4$

In [ ]:
b4 = np.ones(4) / 4.0
dlti4 = signal.dlti(b4, [1])
draw_pzmap(dlti4)

In [ ]:
w, mag = dlti4.freqresp()
plt.plot(w/np.pi, np.abs(mag))

In [ ]:
x = np.array([0, 1, 0, -1] * 5)
y = signal.lfilter(b4, [1], x)

draw_resp_stem(x, y)

In [ ]:
@interact(phase=(0, 90, 5))
def resp(phase=30):
  t = np.arange(16)
  x = np.cos(t * 1/2 * np.pi - phase / 180 * np.pi)
  y = signal.lfilter(b4, [1], x)
  plt.stem(t, x, use_line_collection=True)
  t1 = np.arange(151) / 10
  plt.plot(t1, np.cos(t1 * 1/2 * np.pi - phase / 180 * np.pi), 'y--')
  (markerline, stemlines, baseline) = plt.stem(t, y, linefmt='r', markerfmt='ro', use_line_collection=True)
  markerline.set_markerfacecolor('none')

## Differentiator

$y[n]=x[n]-x[n-1]$

In [ ]:
bd = [1, -1]
x = np.arange(10, dtype='d')  ** 2 
print(x)
y = signal.lfilter(bd, [1], x)
print(y)
z = signal.lfilter(bd, [1], y)
print(z)
z = signal.lfilter(bd, [1], z)
print(z)

In [ ]:
dlti = signal.dlti(bd, [1])
zeros = dlti.zeros
print('Zeros:', zeros)
print('Poles:', dlti.poles)
draw_pzmap(dlti)

In [ ]:
w, mag = dlti.freqresp()
plt.plot(w/np.pi, np.abs(mag))

### Eigenfunction

In [ ]:
x = 2.0 ** np.arange(14)

y = signal.lfilter(bd, [1], x)
z = signal.lfilter(bd, [1], y)
w = signal.lfilter(bd, [1], z)
np.array([x, y, z, w]).T

In [ ]:
t = np.linspace(0, 1,100) * 4 * np.pi
x = np.sin(t)
y = signal.lfilter(bd, [1], x)
plt.plot(t/np.pi, x)
plt.plot(t/np.pi, y/(t[1]-t[0]))

## Recursive

本节内容视频讲解：https://youtu.be/NXnkhopsUnE 国内：[自回归](https://www.bilibili.com/video/BV1FW4y1U7dY)

![IIR](data/iir.png)

$$y[n] = \sum_{i=0}^{L}b[i]x[n-i] - \sum_{i=1}^{M}a[i]y[n-i]$$

### Accumulator, integrator

$y[n] = x[n] + y[n-1]$

$H(z)=\dfrac{1}{1-z^{-1}}$

In [ ]:
dlti = signal.dlti([1], [1, -1])
draw_pzmap(dlti)
print('Zeros:', dlti.zeros)
print('Poles:', dlti.poles)

In [ ]:
b = [1]
a = [1, -1]
x = np.zeros(11)
x[1] = 1
print(x)
y = signal.lfilter(b, a, x=x)
print(y)
z = signal.lfilter(b, a, y)
print(z)
w = signal.lfilter(b, a, z)
print(w)

In [ ]:
w, mag = dlti.freqresp(w=np.linspace(np.pi/1000, np.pi))
plt.plot(w/np.pi, np.abs(mag))

In [ ]:
t, y = dlti.impulse(n=20)
plt.stem(t, y[0], use_line_collection=True)

In [ ]:
t, y = dlti.step(n=20)
plt.stem(t, y[0], use_line_collection=True)

In [ ]:
w3 = signal.lfilter(b, a, w)
plt.plot(z)   # linear
plt.plot(w)   # quadratic
plt.plot(w3)  # 3-order

#### Eigenfunction

In [ ]:
x = 2.0 ** np.arange(11)
y = signal.lfilter(b, a, x, zi=x[0:1])[0]
z = signal.lfilter(b, a, y, zi=y[0:1])[0]
w = signal.lfilter(b, a, z, zi=z[0:1])[0]
np.array([x, y, z, w]).T

In [ ]:
t = np.linspace(0, 1,100) * 4 * np.pi
x = np.sin(t)
y = signal.lfilter(b, a, x)
plt.plot(t/np.pi, x)
plt.plot(t/np.pi, y*(t[1]-t[0]))

模拟反向积分器

![](data/integrator.png)

https://en.wikipedia.org/wiki/Integrator


$$V_\text{out}(t_1) = V_\text{out}(t_0) - \frac{1}{RC} \int_{t_0}^{t_1} V_\text{in}(t) \,dt$$

* R = 1 MΩ
* C = 1 μF
* Vin = 1 V (DC)
* Ir = 1 μA
* Vout = -t V
* Vs = ±15V

那么积十几秒就会饱和。

OPA627
* Vos = 100 μV
* Ib = 1 pA

### Compound interest

$y[n] = x[n] + a y[n-1]$

$H(z)=\dfrac{1}{1-az^{-1}}$

In [ ]:
a = 1.1  # 年投资回报率 10%

dlti = signal.dlti([1], [1, -a])
draw_pzmap(dlti)
print('Zeros:', dlti.zeros)
print('Poles:', dlti.poles)

In [ ]:
t, y = dlti.impulse(n=30)
plt.stem(t, y[0], use_line_collection=True)

In [ ]:
t, y = dlti.step(n=30)
plt.stem(t, y[0], use_line_collection=True)

#### 零状态响应

年息 1.2%，月息 0.1%，每月存 1000。$a = 1+ 1.2\% / 12 = 1.001$

In [ ]:
rate = 1.2
a = 1.0 + rate / 100 / 12

x = np.ones(12) * 1000
y = signal.lfilter([1], [1, -a], x)
print(np.array([x, np.round(y, 2)]).T)

#### 零输入响应

年息 1.2%，月息 0.1%，起始存款 10000。

In [ ]:
rate = 1.2
a = 1.0 + rate / 100 / 12
y0 = 10000.0

x = np.zeros(13) * 0.0
y, yn = signal.lfilter([1], [1, -a], x, zi=[y0])
np.set_printoptions(suppress=True)
print(np.array([np.arange(len(x)), np.round(y, 2)]).T)
# print('yn =', np.round(yn, 2))

#### 完全响应

贷款 20000，年息 3.5%，分 24 期（2 年）偿还，每月还款 864.06。

$p = L\dfrac{c(1+c)^n}{(1+c)^n-1}$, $L=20\,000$, $n = 24$, $c = 3.5\%/12$.

In [ ]:
L = 20000
n = 24
rate = 3.5
c = rate / 100 / 12
p = L * c * np.power(1+c, n) / (np.power(1+c, n) - 1)
print('Monthly payment: %.4f' % p)

In [ ]:
rate = 3.5
a = 1.0 + rate / 100 / 12
y0 = 20000.0

x = np.concatenate(([0.0], np.ones(24) * -864.06))
y, yn = signal.lfilter([1], [1, -a], x, zi=[y0])
np.set_printoptions(suppress=True)
print(np.array([np.arange(len(x)), np.round(y, 2)]).T)
print('yn =', np.round(yn, 2))

### Exponential moving average

$y[n] = ax[n]+(1-a)y[n-1]$

$H(z) = \dfrac{a}{1 - (1-a)z^{-1}}$

证明：对于 x[n] = constant, y[n] 收敛到 constant。

$H(1) = \dfrac{a}{1-(1-a)} = \dfrac{a}{a} = 1$, if $a \ne 0$

In [ ]:
a = 0.8

dlti = signal.dlti([a], [1, a-1])
draw_pzmap(dlti)
print('Zeros:', dlti.zeros)
print('Poles:', dlti.poles)

In [ ]:
@interact(a=(0.1, 0.9, 0.1))
def ema(a):
  dlti = signal.dlti([a], [1, a-1])
  t, imp = dlti.impulse()
  plt.stem(t, imp[0], use_line_collection=True)
  plt.figure(2)
  t, step = dlti.step()
  plt.stem(t, step[0], use_line_collection=True)

In [ ]:
@interact(a=(0.1, 0.9, 0.1))
def ema(a):
  dlti = signal.dlti([a], [1, a-1])
  draw_pzmap(dlti)
  plt.figure(2)
  w, mag = dlti.freqresp()
  plt.plot(w/np.pi, np.abs(mag))
  plt.ylim(0, 1.1)

#### How does TCP calculates RTT

### Fibonacci sequence

$y[n] = y[n-1] + y[n-2]$, with $y[-1] = 1$

$H(z) = \dfrac{1}{1-z^{-1}-z^{-2}}$

In [ ]:
dlti = signal.dlti([1], [1, -1, -1])
draw_pzmap(dlti)
print('Zeros:', dlti.zeros)
print('Poles:', dlti.poles)

In [ ]:
dlti.impulse(n=15)[1]

### Second order digital sinusoidal oscillator

$y[n]=2\cos(w)y[n-1]-y[n-2]$, with $y[-1] = \sin(w)$

$H(z) = \dfrac{1}{1-2\cos(w)z^{-1}+z^{-2}}$

* Digital resonator
https://ccrma.stanford.edu/~jos/pdf/GordonAndSmith86.pdf
* [MIT OCW 6.003](https://ocw.mit.edu/courses/6-003-signals-and-systems-fall-2011/pages/readings/) notes [Chap 6 - The perfect (sine) wave](https://ocw.mit.edu/courses/6-003-signals-and-systems-fall-2011/resources/mit6_003f11_chap6/) using [leapfrog method](https://en.wikipedia.org/wiki/Leapfrog_integration).

In [ ]:
w = np.pi / 10
dlti = signal.dlti([1], [1, -2 * np.cos(w), 1])
draw_pzmap(dlti)
print('Zeros:', dlti.zeros)
print('Poles:', dlti.poles)
print('Poles amp:', np.abs(dlti.poles), "angle:", np.angle(dlti.poles) / np.pi * 180)

In [ ]:
y = dlti.impulse()[1]
plt.plot(y[0])

In [ ]:
# This is numerically instable

fs = 1000
f = 60
w = 2*np.pi*f/fs
c = np.cos(w)
a = [1, -2*c, 1]
n = 101
x = np.zeros(n) * 0.0
y0 = np.sin(w)
y, yn = signal.lfilter([1], a, x, zi=[0, y0])
t = np.arange(n)/fs
plt.plot(t, y)
print(max(y), min(y))

### Summary

单输入单输出（SISO）的 DLTI 系统可由 $\mathbf{a}, \mathbf{b}$ 两个向量，总共 $L+M+1$ 个系数完全定义：

![IIR](data/iir.png)

$$\begin{aligned}
y[n] = &\sum_{i=0}^{L}b[i]x[n-i] - \sum_{i=1}^{M}a[i\,]y[n-i\,] \\
 = & \  b[0]x[n] + b[1]x[n-1] + \cdots + b[L]x[n-L] \\
        &-(a[1]y[n-1]+a[2]y[n-2]+\cdots+a[M]y[n-M])
\end{aligned}$$

**注解**

1. LTI 是 Linear Time-Invariant 的首字母缩写，现在一般统一按字面意思翻译为“线性时不变”，有些比较旧的书上也叫“线性定常”。
2. 离散 LTI，有的书也叫线性移不变 （Linear Shift-Invariant）

## Convolution

本节内容视频讲解：https://youtu.be/79YKlXecF7w 国内：[卷积](https://www.bilibili.com/video/BV1ng41117wB)

For LTI systems:

$Output = Convolve(Input, ImpulseResponse)$

Acoustic Impulse Responses

* [Allen Downey - ThinkDSP](https://github.com/AllenDowney/ThinkDSP)
  * Execrise 10.2 _Simulate the sound of your recording in the space where the impulse response was measured, ..._
* [Allen Downey - Introduction to Digital Signal Processing - PyCon 2018](https://www.youtube.com/watch?v=SrJq2AzXZME)
* [Basic Sound Processing in Python | SciPy 2015 | Allen Downey](https://www.youtube.com/watch?v=0ALKGR0I5MA) 15 min

A room (e.g. concert hall) behaves like a LTI system for sound.
Here _Time-Invariant_ is assumed.

 * [Open Air Library](https://www.openairlib.net/) - acoustic impulse responses
 * [PyRoomAcoustics](https://pyroomacoustics.readthedocs.io/en/pypi-release/index.html) - Room Acoustics Simulation

Librosa

* https://medium.com/@patrickbfuller/librosa-a-python-audio-libary-60014eeaccfb
* https://www.youtube.com/watch?v=MhOdbtPhbLU


In [ ]:
rosa.util.list_examples()

本节的实验会用到本地文件，因此无法在 Colab 上直接运行。

In [ ]:
x, sr = rosa.load('data/poem.ogg')
t = np.arange(len(x)) / sr
plt.plot(t, x)

Audio(x, rate=sr, normalize=False)

In [ ]:
f = rosa.stft(x)
rosa.display.specshow(rosa.amplitude_to_db(np.abs(f)), sr=sr, y_axis='hz', x_axis='s')
plt.colorbar()

In [ ]:
spec = rosa.feature.melspectrogram(y=x, sr=sr)
db_spec = rosa.power_to_db(spec, ref=np.max)
rosa.display.specshow(db_spec,y_axis='mel', x_axis='s', sr=sr)
# plt.colorbar()

### Innocent Railway Tunnel

https://www.openair.hosted.york.ac.uk/?page_id=525

这是一个过去的火车隧道的冲击响应。

In [ ]:
h1, sr = rosa.load('data/middle_tunnel_1way_mono.flac')

rosa.display.waveshow(y=h1, sr=sr)
Audio(h1, rate=sr, normalize=False)

In [ ]:
y1 = signal.oaconvolve(x, h1, mode='full')
print("%.3f sec" % (len(y1)/sr))
Audio(y1, rate=sr, normalize=True)

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(121)
f1 = rosa.stft(y1[0:sr*9])
rosa.display.specshow(rosa.amplitude_to_db(np.abs(f1)), sr=sr, y_axis='hz', x_axis='s')
plt.title('Convoluted')

plt.subplot(122)
rosa.display.specshow(rosa.amplitude_to_db(np.abs(f)), sr=sr, y_axis='hz', x_axis='s')
plt.title('Original')

**练习**：用英国 York 大学 Central Hall 的冲击响应重做以上实验。

https://www.openair.hosted.york.ac.uk/?page_id=435

### Deconvolution

https://en.wikipedia.org/wiki/Deconvolution

One of Prof. Alan V. Oppenheim's faviourite examples: Digital restoration of Enrico Caruso's recordings by Thomas Stockham. 1975 paper: "Blind deconvolution through Digital Signal Processing".

* [MIT RES.6-007 Signals ans Systems, 1987](https://youtu.be/KJnAy6hzetw?t=760)
* [MIT RES.6-008 Digital Signal Processing, 1975](https://youtu.be/rkvEM5Y3N60?t=338)
* [MITx 6.341x Discrete-Time Signal Processing, 2016](https://learning.edx.org/course/course-v1:MITx+6.341x_2+2T2016/block-v1:MITx+6.341x_2+2T2016+type@sequential+block@Enrichment_lecture_How_Caruso_Lost_His_Orchestra/block-v1:MITx+6.341x_2+2T2016+type@vertical+block@Enrichment_Lecture_2_Video_3)



## Summary

DLTI 系统模型之间的相互转换

![DLTI](data/dlti.png)

$$\begin{aligned}
y[n] &= \sum_{i=0}^{L}b_ix[n-i\,] - \sum_{i=1}^{M}a_iy[n-i\,] \\
     &=  \  b_0x[n] + b_1x[n-1] + \cdots + b_Lx[n-L] \\
     & \quad   -(a_1y[n-1]+a_2y[n-2]+\cdots+a_My[n-M])\\[3mm]
H(z) &= \frac{\sum_{i=0}^{L}b_iz^{-i}}{1+ \sum_{i=1}^{M}a_iz^{-i}} \\
     &= \frac{b_0 + b_1z^{-1}+b_2z^{-2}+\cdots+b_Lz^{-L}}{1+a_1z^{-1}+a_2z^{-2}+\cdots+a_Mz^{-M}}\\[3mm]
H(z) &= b_0\frac{\prod_{i=1}^L(1-z_iz^{-1})}{\prod_{i=1}^M(1-p_iz^{-1})} \\
     &=b_0\frac{(1-z_1z^-1)(1-z_2z^{-1})\cdots(1-z_Lz^{-1})}{(1-p_1z^-1)(1-p_2z^{-1})\cdots(1-z_Mz^{-1})}
\end{aligned}$$

本套实验没有使用 $z$ 变换分析 DLTI 系统，这种方法在一般的教材上都会介绍，我就不费口舌了。比如
https://web.eecs.umich.edu/~fessler/course/451/l/pdf/c3.pdf